# 제주 핫플 - 인스타 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
chromedriver = 'C:\\MyWorkspace\\02.DataAnalysis\\05.web\\chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

## 인스타그램 접속 & 로그인

In [3]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(2)

In [4]:
with open('stg_password.txt') as f:
    password = f.read()

In [5]:
email = 'shin.jae.ho.eh'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [6]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [7]:
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(3)
except:
    pass

In [8]:
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

## 제주도맛집 검색

In [ ]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'

In [ ]:
driver.get(url)
time.sleep(8)

## 3. 첫 게시물 열기

In [ ]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(2)

## 4. 게시글 열기

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
import unicodedata
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ''
content

## 5. 본문 내용에서 해쉬태그(#) 가져오기, '#[^\s#,\\]+'

In [ ]:
import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags

## 작성일자 정보 가져오기

In [ ]:
date = soup.select_one('time._1o9PC.Nzb55')['datetime'][:10]
date

## 좋아요 수 가져오기

In [ ]:
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

In [ ]:
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

In [ ]:
row = [content, date, like, place, tags]
row

## 다음 게시글

In [ ]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [ ]:
# 6을 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

## 6. 여러 게시글

In [9]:
import unicodedata
import re
from tqdm.notebook import tqdm

In [10]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, tags, date, like, place]

In [13]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

KeyboardInterrupt: 

In [ ]:
results = []
for _ in tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

In [ ]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

In [ ]:
df.tail()

In [ ]:
df.to_csv('data/제주도맛집.csv', index=False)

## 여러태그 여러게시글

In [ ]:
for keyword in ['제주도맛집','제주맛집','제주도관광','제주여행']:
    search_url = 'https://www.instagram.com/explore/tags/'
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(10)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(2)

    results = []
    for _ in tqdm(range(100)):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)

    df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
    df.to_csv(f'data/{keyword}.csv', index=False)

    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(5)